# Import Relevant Libraries and Load in Merged Datasets

In [62]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [64]:
df = pd.read_csv('data/merged_data.csv')

/Users/jamiecarnevale/miniconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,21,30,63,97,99,118,119,120,122,126,127,128,131,133,136,137,139,149) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Data Exploration and Cleaning

In [65]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 150 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     731907 non-null  int64  
 1   PERSON_ID                      731907 non-null  object 
 2   PERSON_TYPE                    731907 non-null  object 
 3   CRASH_RECORD_ID                731907 non-null  object 
 4   RD_NO_x                        727570 non-null  object 
 5   VEHICLE_ID_x                   725151 non-null  float64
 6   CRASH_DATE_x                   731907 non-null  object 
 7   SEAT_NO                        60155 non-null   float64
 8   CITY                           460748 non-null  object 
 9   STATE                          459075 non-null  object 
 10  ZIPCODE                        422795 non-null  object 
 11  SEX                            727658 non-null  object 
 12  AGE                          

In [66]:
df.iloc[:,100:120].head()

,TOWED_BY,TOWED_TO,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO
0,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRONT,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIDE-RIGHT-FRONT,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,FRONT-LEFT,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,UNKNOWN,NaN,NaN,NaN


### let's drop all columns that have less than 20k nonnull values

In [67]:
df.columns[df.notna().sum() < 20000]

Index(['EMS_RUN_NO', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY',
       'PEDPEDAL_LOCATION', 'BAC_RESULT VALUE', 'CELL_PHONE_USE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'CMRC_VEH_I', 'FIRE_I',
       'EXCEED_SPEED_LIMIT_I', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO',
       'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',
       'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO',
       'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I',
       'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH',
       'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT', 'VEHICLE_CONFIG',
       'CARGO_BODY_TYPE', 'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I',
       'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS'],
      dtype='object')

In [68]:
colstodrop = df.columns[df.notna().sum() < 250000]

In [69]:
df1 = df.drop(colstodrop, axis = 1)

In [70]:
df1['PERSON_TYPE'].value_counts()

DRIVER                 665215
PASSENGER               60155
BICYCLE                  3524
PEDESTRIAN               2403
NON-MOTOR VEHICLE         464
NON-CONTACT VEHICLE       146
Name: PERSON_TYPE, dtype: int64

In [71]:
df1

,Unnamed: 0,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO_x,VEHICLE_ID_x,CRASH_DATE_x,CITY,STATE,ZIPCODE,...,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT
0,0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,CHICAGO,IL,60651,...,CIVIC,IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,1.0,FRONT
1,1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,CHICAGO,IL,60620,...,ENCORE,IL,NaN,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,1.0,SIDE-RIGHT-FRONT
2,2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,...,UNKNOWN,XX,NaN,NONE,PASSENGER,PERSONAL,E,BACKING,1.0,REAR
3,3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,...,GRAND CARAVAN,IL,2012.0,NONE,PASSENGER,NOT IN USE,N,PARKED,0.0,FRONT-LEFT
4,4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,...,UNKNOWN,XX,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN,UNKNOWN/NA,1.0,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731902,731902,O1559700,DRIVER,462b5811f1ff74d554aaaa269233f41a68635bb3d1e52c...,JG228070,1483547.0,04/18/2023 11:32:00 AM,ROUND LAKE,IL,60073,...,PATRIOT,IL,2015.0,UNKNOWN,PASSENGER,PERSONAL,S,TURNING ON RED,2.0,SIDE-RIGHT
731903,731903,O1560774,DRIVER,dc203cc2a56651a63090b1817fbd40e8905efa97031346...,JG234213,1484605.0,04/18/2023 06:45:00 AM,NaN,NaN,NaN,...,DURANGO,IL,2017.0,UNKNOWN,SPORT UTILITY VEHICLE (SUV),PERSONAL,S,PASSING/OVERTAKING,1.0,FRONT-LEFT-CORNER
731904,731904,O1561819,DRIVER,aa8ada359fd31d545b59706ded379ea1193d43419f76e3...,JG236026,1485616.0,04/18/2023 07:12:00 PM,NaN,NaN,NaN,...,OTHER (EXPLAIN IN NARRATIVE),IL,2005.0,NONE,PICKUP,OTHER,E,STRAIGHT AHEAD,1.0,FRONT-RIGHT-CORNER
731905,731905,O1563799,DRIVER,4050f4309ea3c72d42fbe9b22bb9933d9268c543147e3a...,JG228786,1487508.0,04/18/2023 06:20:00 PM,CHICAGO,IL,60616,...,RAV4,IL,2018.0,UNKNOWN,SPORT UTILITY VEHICLE (SUV),PERSONAL,W,TURNING RIGHT,1.0,FRONT-RIGHT-CORNER


In [72]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 76 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     731907 non-null  int64  
 1   PERSON_ID                      731907 non-null  object 
 2   PERSON_TYPE                    731907 non-null  object 
 3   CRASH_RECORD_ID                731907 non-null  object 
 4   RD_NO_x                        727570 non-null  object 
 5   VEHICLE_ID_x                   725151 non-null  float64
 6   CRASH_DATE_x                   731907 non-null  object 
 7   CITY                           460748 non-null  object 
 8   STATE                          459075 non-null  object 
 9   ZIPCODE                        422795 non-null  object 
 10  SEX                            727658 non-null  object 
 11  AGE                            434077 non-null  float64
 12  DRIVERS_LICENSE_STATE         

In [73]:
df1.head()

,Unnamed: 0,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO_x,VEHICLE_ID_x,CRASH_DATE_x,CITY,STATE,ZIPCODE,...,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT
0,0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,CHICAGO,IL,60651,...,CIVIC,IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,1.0,FRONT
1,1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,CHICAGO,IL,60620,...,ENCORE,IL,NaN,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,1.0,SIDE-RIGHT-FRONT
2,2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,...,UNKNOWN,XX,NaN,NONE,PASSENGER,PERSONAL,E,BACKING,1.0,REAR
3,3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,...,GRAND CARAVAN,IL,2012.0,NONE,PASSENGER,NOT IN USE,N,PARKED,0.0,FRONT-LEFT
4,4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,...,UNKNOWN,XX,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN,UNKNOWN/NA,1.0,UNKNOWN


In [74]:
df1['DRIVER_VISION'].value_counts()

UNKNOWN                   405885
NOT OBSCURED              245541
OTHER                       8286
MOVING VEHICLES             4297
PARKED VEHICLES             3043
WINDSHIELD (WATER/ICE)      1894
BLINDED - SUNLIGHT          1261
TREES, PLANTS                340
BUILDINGS                    258
BLINDED - HEADLIGHTS          89
BLOWING MATERIALS             63
HILLCREST                     61
EMBANKMENT                    44
SIGNBOARD                     20
Name: DRIVER_VISION, dtype: int64

In [75]:
df1['MOST_SEVERE_INJURY'].value_counts()

NO INDICATION OF INJURY     632576
NONINCAPACITATING INJURY     56080
REPORTED, NOT EVIDENT        29971
INCAPACITATING INJURY        12470
FATAL                          799
Name: MOST_SEVERE_INJURY, dtype: int64

In [76]:
df1['VEHICLE_DEFECT'].value_counts()

UNKNOWN             398828
NONE                305305
OTHER                 4822
BRAKES                3524
TIRES                  632
STEERING               597
WHEELS                 329
SUSPENSION             222
ENGINE/MOTOR           129
FUEL SYSTEM            117
LIGHTS                  59
WINDOWS                 52
CARGO                   35
SIGNALS                 22
RESTRAINT SYSTEM        17
TRAILER COUPLING        15
EXHAUST                 11
Name: VEHICLE_DEFECT, dtype: int64

In [77]:
df1['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

UNABLE TO DETERMINE                                                                 282207
FAILING TO YIELD RIGHT-OF-WAY                                                        80260
FOLLOWING TOO CLOSELY                                                                72567
NOT APPLICABLE                                                                       38452
IMPROPER OVERTAKING/PASSING                                                          35747
FAILING TO REDUCE SPEED TO AVOID CRASH                                               31048
IMPROPER BACKING                                                                     29473
IMPROPER LANE USAGE                                                                  26774
IMPROPER TURNING/NO SIGNAL                                                           24259
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  24041
DISREGARDING TRAFFIC SIGNALS                                                         14223

In [78]:
df1['LOCATION'].value_counts()

POINT (-87.905309125103 41.976201139024)    1161
POINT (-87.619928173678 41.900958919109)     674
POINT (-87.580147768689 41.791420282098)     527
POINT (-87.585971992965 41.751460603167)     512
POINT (-87.585275565077 41.722257273006)     400
                                            ... 
POINT (-87.70989030694 41.822624359327)        1
POINT (-87.674513447715 41.747609302941)       1
POINT (-87.806081541165 41.923260716676)       1
POINT (-87.736742235417 41.791581352507)       1
POINT (-87.616402310151 41.886166957197)       1
Name: LOCATION, Length: 279315, dtype: int64

In [79]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 76 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     731907 non-null  int64  
 1   PERSON_ID                      731907 non-null  object 
 2   PERSON_TYPE                    731907 non-null  object 
 3   CRASH_RECORD_ID                731907 non-null  object 
 4   RD_NO_x                        727570 non-null  object 
 5   VEHICLE_ID_x                   725151 non-null  float64
 6   CRASH_DATE_x                   731907 non-null  object 
 7   CITY                           460748 non-null  object 
 8   STATE                          459075 non-null  object 
 9   ZIPCODE                        422795 non-null  object 
 10  SEX                            727658 non-null  object 
 11  AGE                            434077 non-null  float64
 12  DRIVERS_LICENSE_STATE         

### lets further hone in on the relevant columns

In [80]:
df2 = df1[[
    'PERSON_TYPE',
    'SEX',
    'EJECTION',
    'INJURY_CLASSIFICATION',
    'DRIVER_VISION',
    'PHYSICAL_CONDITION',
    'POSTED_SPEED_LIMIT',
    'TRAFFIC_CONTROL_DEVICE',
    'DEVICE_CONDITION',
    'WEATHER_CONDITION',
    'LIGHTING_CONDITION',
    'FIRST_CRASH_TYPE',
    'TRAFFICWAY_TYPE',
    'ALIGNMENT',
    'ROADWAY_SURFACE_COND',
    'ROAD_DEFECT',
    'REPORT_TYPE',
    'CRASH_TYPE',
    'DAMAGE',
    'DATE_POLICE_NOTIFIED',
    'PRIM_CONTRIBUTORY_CAUSE',
    'STREET_NO',
    'STREET_DIRECTION',
    'STREET_NAME',
    'BEAT_OF_OCCURRENCE',
    'NUM_UNITS',
    'MOST_SEVERE_INJURY',
    'INJURIES_TOTAL',
    'INJURIES_FATAL',
    'INJURIES_INCAPACITATING',
    'INJURIES_NON_INCAPACITATING',
    'INJURIES_REPORTED_NOT_EVIDENT',
    'INJURIES_NO_INDICATION',
    'INJURIES_UNKNOWN',
    'CRASH_HOUR',
    'CRASH_DAY_OF_WEEK',
    'CRASH_MONTH',
    'LATITUDE',
    'LONGITUDE',
    'LOCATION',
    'CRASH_DATE',
    'UNIT_TYPE',
    'MAKE',
    'MODEL',
    'LIC_PLATE_STATE',
    'VEHICLE_YEAR',
    'VEHICLE_DEFECT',
    'VEHICLE_TYPE',
    'VEHICLE_USE',
    'TRAVEL_DIRECTION',
    'MANEUVER',
    'OCCUPANT_CNT',
    'FIRST_CONTACT_POINT',
    'AGE',
    'BAC_RESULT',
    'SAFETY_EQUIPMENT',
    'AIRBAG_DEPLOYED'
]]



In [81]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 57 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    731907 non-null  object 
 1   SEX                            727658 non-null  object 
 2   EJECTION                       729172 non-null  object 
 3   INJURY_CLASSIFICATION          731723 non-null  object 
 4   DRIVER_VISION                  671082 non-null  object 
 5   PHYSICAL_CONDITION             671290 non-null  object 
 6   POSTED_SPEED_LIMIT             731907 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         731907 non-null  object 
 8   DEVICE_CONDITION               731907 non-null  object 
 9   WEATHER_CONDITION              731907 non-null  object 
 10  LIGHTING_CONDITION             731907 non-null  object 
 11  FIRST_CRASH_TYPE               731907 non-null  object 
 12  TRAFFICWAY_TYPE               

## lets do some quick cleaning up of outliers

In [82]:
df2['VEHICLE_YEAR'].value_counts()
#looks like there are some vehicle years that dont make sense

2015.0    36281
2016.0    33811
2014.0    32851
2017.0    32795
2013.0    30706
          ...  
3016.0        1
3023.0        1
3027.0        1
5004.0        1
2054.0        1
Name: VEHICLE_YEAR, Length: 169, dtype: int64

In [83]:
#quickly cleaning up the outliers
df2 = df2[(df['VEHICLE_YEAR'] <= 2024) & (df['VEHICLE_YEAR'] >= 1930)]

In [84]:
df2['VEHICLE_YEAR'].value_counts().tail(20)
#looks clean

1977.0    16
1971.0    14
1974.0    11
1969.0    11
1973.0     9
1965.0     6
1964.0     6
1963.0     5
1966.0     5
1968.0     5
1967.0     3
1953.0     2
1952.0     2
1962.0     1
1961.0     1
1960.0     1
1958.0     1
1957.0     1
1954.0     1
1941.0     1
Name: VEHICLE_YEAR, dtype: int64

In [85]:
df2['AGE'].value_counts().tail()
#need to clean up some of these negative values that can't exist

 109.0    2
 102.0    2
-1.0      1
 110.0    1
 108.0    1
Name: AGE, dtype: int64

In [86]:
df2 = df2[(df['AGE'] <= 110) & (df['AGE'] >= 0)]
#this should do it

<ipython-input-86-2ba7b13e0de3>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[(df['AGE'] <= 110) & (df['AGE'] >= 0)]


In [87]:
df2['AGE'].value_counts().tail()
#better

107.0    3
102.0    2
109.0    2
108.0    1
110.0    1
Name: AGE, dtype: int64

In [88]:
df2['CRASH_DAY_OF_WEEK'].value_counts()
#looks clean already

6    65469
5    58372
4    57633
3    57587
7    56390
2    54284
1    45259
Name: CRASH_DAY_OF_WEEK, dtype: int64

In [89]:
df2.groupby('MOST_SEVERE_INJURY').mean()
#just curious

,POSTED_SPEED_LIMIT,STREET_NO,BEAT_OF_OCCURRENCE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,VEHICLE_YEAR,OCCUPANT_CNT,AGE
MOST_SEVERE_INJURY,,,,,,,,,,,,,,,,,,,
FATAL,29.818913,4334.507042,1041.156942,2.062374,1.937626,1.096579,0.287726,0.460765,0.092555,0.730382,0.0,12.156942,4.054326,6.692153,41.823726,-87.670369,2008.941650,1.593561,38.066398
INCAPACITATING INJURY,30.046626,3850.259166,1158.003985,2.138417,1.643597,0.000000,1.206562,0.341259,0.095776,1.193544,0.0,12.792508,4.081031,6.733927,41.844281,-87.665615,2010.058980,1.461211,37.642667
NO INDICATION OF INJURY,28.491539,3618.968111,1276.278245,2.004651,0.000000,0.000000,0.000000,0.000000,0.000000,2.241901,0.0,13.314517,4.150524,6.544793,41.860576,-87.674103,2010.644449,1.181486,38.923103
NONINCAPACITATING INJURY,29.898719,3808.405324,1175.471663,2.126530,1.466888,0.000000,0.000000,1.381534,0.085354,1.365092,0.0,12.972521,4.096163,6.686463,41.842549,-87.666270,2010.441639,1.409842,37.185148
"REPORTED, NOT EVIDENT",29.764955,3757.728205,1213.592640,2.121427,1.349275,0.000000,0.000000,0.000000,1.349275,1.582227,0.0,13.254900,4.128267,6.627909,41.842563,-87.664758,2010.460698,1.364434,37.388628


In [90]:
df2.columns

Index(['PERSON_TYPE', 'SEX', 'EJECTION', 'INJURY_CLASSIFICATION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION', 'POSTED_SPEED_LIMIT',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE',
       'CRASH_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION',
       'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'MOST_SEVERE_INJURY',
       'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR',
       'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'CRASH_DATE', 'UNIT_TYPE', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
       'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRE

In [91]:
df2['MAKE'].value_counts()

CHEVROLET                                49842
FORD                                     47011
NISSAN                                   33041
HONDA                                    30642
TOYOTA MOTOR COMPANY, LTD.               26856
                                         ...  
ROADLINER MANUFACTURING DIVISION             1
IMPERIAL INDUSTRIES                          1
E Z HAUL, DIVISION PARKHURST MFG. CO.        1
ROKON                                        1
XPLORER ENTERPRISES OF CONSTANTINE           1
Name: MAKE, Length: 794, dtype: int64

In [92]:
df2['BAC_RESULT'].value_counts()

TEST NOT OFFERED                   354396
TEST REFUSED                         5232
TEST TAKEN                           1686
TEST PERFORMED, RESULTS UNKNOWN      1474
Name: BAC_RESULT, dtype: int64

In [93]:
df2.shape

(394994, 57)

# Creating Target Prediction Column

In [160]:
df2['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

FAILING TO YIELD RIGHT-OF-WAY                                                       45186
FOLLOWING TOO CLOSELY                                                               38212
FAILING TO REDUCE SPEED TO AVOID CRASH                                              16512
IMPROPER OVERTAKING/PASSING                                                         14938
IMPROPER BACKING                                                                    14032
IMPROPER TURNING/NO SIGNAL                                                          13324
IMPROPER LANE USAGE                                                                 12780
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                 12240
WEATHER                                                                              7626
DISREGARDING TRAFFIC SIGNALS                                                         7161
DISTRACTION - FROM INSIDE VEHICLE                                                    3608
DISREGARDI

In [161]:
len(df2.loc[(df2['PRIM_CONTRIBUTORY_CAUSE'] =='UNABLE TO DETERMINE') | (df2['PRIM_CONTRIBUTORY_CAUSE'] == 'NOT APPLICABLE')])/df2.shape[0]

0.0

In [168]:
target_dict = {
    'reckless driving': [
        'FOLLOWING TOO CLOSELY', 
        'IMPROPER OVERTAKING/PASSING', 
        'EXCEEDING SAFE SPEED FOR CONDITIONS',
        'EXCEEDING AUTHORIZED SPEED LIMIT', 
        'IMPROPER TURNING/NO SIGNAL', 
          
        'IMPROPER LANE USAGE', 
         
        'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
         
        'DRIVING ON WRONG SIDE/WRONG WAY'
        'CELL PHONE USE OTHER THAN TEXTING', 
        'TEXTING',
        'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)', 
        'DISTRACTION - FROM INSIDE VEHICLE'
    ],

    'nondefensive driving' :
        ['IMPROPER BACKING',
        'FAILING TO REDUCE SPEED TO AVOID CRASH',
        'TURNING RIGHT ON RED'
        
        ],
    
    
    'missed or misunderstood traffic signs': [
        'DISREGARDING YIELD SIGN', 
        'FAILING TO YIELD RIGHT-OF-WAY', 
        'DISREGARDING TRAFFIC SIGNALS', 
        'DISREGARDING OTHER TRAFFIC SIGNS',
        'DISREGARDING ROAD MARKINGS', 
        'DISREGARDING STOP SIGN'
    ],

    'issue with the driver or vehicle': [
        'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE', 
        'PHYSICAL CONDITION OF DRIVER', 
        'EQUIPMENT - VEHICLE CONDITION', 
        'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',
        'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)'
    ],

    'external variable': [
        'WEATHER', 
        'DISTRACTION - FROM OUTSIDE VEHICLE',
        'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)', 
        'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST',
        'ROAD CONSTRUCTION/MAINTENANCE', 
        'ROAD ENGINEERING/SURFACE/MARKING DEFECTS', 
        'ANIMAL', 
        'OBSTRUCTED CROSSWALKS',
    ],

    'drop': [
        'PASSING STOPPED SCHOOL BUS', 
        'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT',
        'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 
        'RELATED TO BUS STOP'
    ]
}


In [169]:
target_dict

{'reckless driving': ['FOLLOWING TOO CLOSELY',
  'IMPROPER OVERTAKING/PASSING',
  'EXCEEDING SAFE SPEED FOR CONDITIONS',
  'EXCEEDING AUTHORIZED SPEED LIMIT',
  'IMPROPER TURNING/NO SIGNAL',
  'IMPROPER LANE USAGE',
  'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
  'DRIVING ON WRONG SIDE/WRONG WAYCELL PHONE USE OTHER THAN TEXTING',
  'TEXTING',
  'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',
  'DISTRACTION - FROM INSIDE VEHICLE'],
 'nondefensive driving': ['IMPROPER BACKING',
  'FAILING TO REDUCE SPEED TO AVOID CRASH',
  'TURNING RIGHT ON RED'],
 'missed or misunderstood traffic signs': ['DISREGARDING YIELD SIGN',
  'FAILING TO YIELD RIGHT-OF-WAY',
  'DISREGARDING TRAFFIC SIGNALS',
  'DISREGARDING OTHER TRAFFIC SIGNS',
  'DISREGARDING ROAD MARKINGS',
  'DISREGARDING STOP SIGN'],
 'issue with the driver or vehicle': ['DRIVING SKILLS/KNOWLEDGE/EXPERIENCE',
  'PHYSICAL CONDITION OF DRIVER',
  'EQUIPMENT - VEHICLE CONDI

In [170]:
new_dict = {value: key for key in target_dict for value in target_dict[key]}

In [171]:
df2['target_mapped'] = df2['PRIM_CONTRIBUTORY_CAUSE']

In [172]:
df2['target_mapped'] = df2['target_mapped'].map(new_dict)

In [173]:
df2['target_mapped'].value_counts()

reckless driving                         87610
missed or misunderstood traffic signs    57137
nondefensive driving                     30800
issue with the driver or vehicle         21942
external variable                        16499
Name: target_mapped, dtype: int64

In [174]:
df2 = df2[df2['target_mapped'] != 'drop']

# Further Cleaning (including dropping null values)

In [175]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213988 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    213988 non-null  object 
 1   SEX                            213988 non-null  object 
 2   EJECTION                       213988 non-null  object 
 3   INJURY_CLASSIFICATION          213988 non-null  object 
 4   DRIVER_VISION                  213988 non-null  object 
 5   PHYSICAL_CONDITION             213988 non-null  object 
 6   POSTED_SPEED_LIMIT             213988 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         213988 non-null  object 
 8   DEVICE_CONDITION               213988 non-null  object 
 9   WEATHER_CONDITION              213988 non-null  object 
 10  LIGHTING_CONDITION             213988 non-null  object 
 11  FIRST_CRASH_TYPE               213988 non-null  object 
 12  TRAFFICWAY_TYPE              

In [176]:
df2 = df2.dropna()

In [177]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213988 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    213988 non-null  object 
 1   SEX                            213988 non-null  object 
 2   EJECTION                       213988 non-null  object 
 3   INJURY_CLASSIFICATION          213988 non-null  object 
 4   DRIVER_VISION                  213988 non-null  object 
 5   PHYSICAL_CONDITION             213988 non-null  object 
 6   POSTED_SPEED_LIMIT             213988 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         213988 non-null  object 
 8   DEVICE_CONDITION               213988 non-null  object 
 9   WEATHER_CONDITION              213988 non-null  object 
 10  LIGHTING_CONDITION             213988 non-null  object 
 11  FIRST_CRASH_TYPE               213988 non-null  object 
 12  TRAFFICWAY_TYPE              

In [178]:
df2.head()

,PERSON_TYPE,SEX,EJECTION,INJURY_CLASSIFICATION,DRIVER_VISION,PHYSICAL_CONDITION,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,...,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT,AGE,BAC_RESULT,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,target_mapped
12,DRIVER,F,NONE,NO INDICATION OF INJURY,NOT OBSCURED,NORMAL,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,...,PERSONAL,NE,MERGING,1.0,FRONT-LEFT-CORNER,34.0,TEST NOT OFFERED,SAFETY BELT USED,DID NOT DEPLOY,reckless driving
18,DRIVER,M,NONE,NO INDICATION OF INJURY,NOT OBSCURED,UNKNOWN,15,NO CONTROLS,NO CONTROLS,CLEAR,...,PERSONAL,W,STRAIGHT AHEAD,1.0,FRONT-RIGHT-CORNER,22.0,TEST NOT OFFERED,SAFETY BELT USED,DID NOT DEPLOY,reckless driving
37,DRIVER,M,NONE,NO INDICATION OF INJURY,UNKNOWN,NORMAL,30,NO CONTROLS,NO CONTROLS,CLEAR,...,PERSONAL,N,PASSING/OVERTAKING,1.0,SIDE-RIGHT-FRONT,22.0,TEST NOT OFFERED,USAGE UNKNOWN,"DEPLOYED, FRONT",reckless driving
38,DRIVER,M,NONE,NO INDICATION OF INJURY,NOT OBSCURED,NORMAL,45,NO CONTROLS,NO CONTROLS,CLEAR,...,PERSONAL,N,SLOW/STOP IN TRAFFIC,2.0,REAR,61.0,TEST NOT OFFERED,SAFETY BELT USED,DID NOT DEPLOY,reckless driving
42,DRIVER,M,NONE,NO INDICATION OF INJURY,NOT OBSCURED,NORMAL,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,...,PERSONAL,N,TURNING LEFT,1.0,FRONT-LEFT-CORNER,28.0,TEST NOT OFFERED,SAFETY BELT USED,"DEPLOYED, COMBINATION",missed or misunderstood traffic signs


In [179]:
for column in df2.columns:
    print(f"Column: {column}")
    print(df2[column].value_counts())
    print("\n")


Column: PERSON_TYPE
DRIVER                 213975
NON-CONTACT VEHICLE        11
NON-MOTOR VEHICLE           2
Name: PERSON_TYPE, dtype: int64


Column: SEX
M    133526
F     80245
X       217
Name: SEX, dtype: int64


Column: EJECTION
NONE                  209774
UNKNOWN                 3408
TOTALLY EJECTED          504
PARTIALLY EJECTED        159
TRAPPED/EXTRICATED       143
Name: EJECTION, dtype: int64


Column: INJURY_CLASSIFICATION
NO INDICATION OF INJURY     198868
NONINCAPACITATING INJURY      9159
REPORTED, NOT EVIDENT         4100
INCAPACITATING INJURY         1755
FATAL                          106
Name: INJURY_CLASSIFICATION, dtype: int64


Column: DRIVER_VISION
NOT OBSCURED              130027
UNKNOWN                    72131
OTHER                       4499
MOVING VEHICLES             2817
PARKED VEHICLES             2066
WINDSHIELD (WATER/ICE)      1163
BLINDED - SUNLIGHT           769
TREES, PLANTS                209
BUILDINGS                    155
BLINDED - HEADLIGHTS 

NO INDICATION OF INJURY     179187
NONINCAPACITATING INJURY     19679
REPORTED, NOT EVIDENT        10800
INCAPACITATING INJURY         4133
FATAL                          189
Name: MOST_SEVERE_INJURY, dtype: int64


Column: INJURIES_TOTAL
0.0     179187
1.0      25237
2.0       6660
3.0       1843
4.0        669
5.0        248
6.0         94
7.0         23
8.0         15
9.0          4
11.0         3
10.0         2
12.0         1
21.0         1
15.0         1
Name: INJURIES_TOTAL, dtype: int64


Column: INJURIES_FATAL
0.0    213799
1.0       177
2.0        12
Name: INJURIES_FATAL, dtype: int64


Column: INJURIES_INCAPACITATING
0.0    209818
1.0      3592
2.0       434
3.0       102
4.0        29
5.0        11
6.0         2
Name: INJURIES_INCAPACITATING, dtype: int64


Column: INJURIES_NON_INCAPACITATING
0.0     193406
1.0      15637
2.0       3495
3.0        950
4.0        321
5.0        117
6.0         36
7.0         12
8.0          6
11.0         2
9.0          2
12.0         1
15.0 

FRONT                 70095
SIDE-RIGHT            19594
FRONT-RIGHT           16370
FRONT-RIGHT-CORNER    16255
FRONT-LEFT            14545
FRONT-LEFT-CORNER     13457
REAR                  11107
SIDE-LEFT              8464
OTHER                  7784
REAR-LEFT              5131
UNKNOWN                4956
REAR-RIGHT             4831
TOTAL (ALL AREAS)      3755
REAR-RIGHT-CORNER      3023
NONE                   2685
REAR-LEFT-CORNER       2470
SIDE-RIGHT-REAR        2045
SIDE-RIGHT-FRONT       1856
SIDE-LEFT-REAR         1685
SIDE-LEFT-FRONT        1492
ROOF                   1087
UNDER CARRIAGE          896
TOP                     405
Name: FIRST_CONTACT_POINT, dtype: int64


Column: AGE
25.0     7017
27.0     6741
24.0     6711
26.0     6689
23.0     6636
         ... 
100.0       3
101.0       2
98.0        1
108.0       1
104.0       1
Name: AGE, Length: 105, dtype: int64


Column: BAC_RESULT
TEST NOT OFFERED                   207821
TEST REFUSED                         3656
TEST T

In [180]:
df2['DAMAGE'].value_counts()

OVER $1,500      142039
$501 - $1,500     51107
$500 OR LESS      20842
Name: DAMAGE, dtype: int64

In [181]:
df2.groupby('MOST_SEVERE_INJURY').count()

,PERSON_TYPE,SEX,EJECTION,INJURY_CLASSIFICATION,DRIVER_VISION,PHYSICAL_CONDITION,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,...,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT,AGE,BAC_RESULT,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,target_mapped
MOST_SEVERE_INJURY,,,,,,,,,,,,,,,,,,,,,
FATAL,189,189,189,189,189,189,189,189,189,189,...,189,189,189,189,189,189,189,189,189,189
INCAPACITATING INJURY,4133,4133,4133,4133,4133,4133,4133,4133,4133,4133,...,4133,4133,4133,4133,4133,4133,4133,4133,4133,4133
NO INDICATION OF INJURY,179187,179187,179187,179187,179187,179187,179187,179187,179187,179187,...,179187,179187,179187,179187,179187,179187,179187,179187,179187,179187
NONINCAPACITATING INJURY,19679,19679,19679,19679,19679,19679,19679,19679,19679,19679,...,19679,19679,19679,19679,19679,19679,19679,19679,19679,19679
"REPORTED, NOT EVIDENT",10800,10800,10800,10800,10800,10800,10800,10800,10800,10800,...,10800,10800,10800,10800,10800,10800,10800,10800,10800,10800


In [182]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213988 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    213988 non-null  object 
 1   SEX                            213988 non-null  object 
 2   EJECTION                       213988 non-null  object 
 3   INJURY_CLASSIFICATION          213988 non-null  object 
 4   DRIVER_VISION                  213988 non-null  object 
 5   PHYSICAL_CONDITION             213988 non-null  object 
 6   POSTED_SPEED_LIMIT             213988 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         213988 non-null  object 
 8   DEVICE_CONDITION               213988 non-null  object 
 9   WEATHER_CONDITION              213988 non-null  object 
 10  LIGHTING_CONDITION             213988 non-null  object 
 11  FIRST_CRASH_TYPE               213988 non-null  object 
 12  TRAFFICWAY_TYPE              

# cleaning up the make and vehicle type

In [183]:
make_counts = df2['MAKE'].value_counts()
over_50 = make_counts[make_counts > 50].index
df2 = df2[df2['MAKE'].isin(over_50)]

In [184]:
df2['MAKE'].value_counts()

CHEVROLET                            27358
TOYOTA                               26193
FORD                                 25301
NISSAN                               18296
HONDA                                17191
                                     ...  
INTERNATIONAL COACH MANUFACTURING       56
MOTOR COACH INDUSTRIES, INC.            55
MASERATI                                55
BLUE BIRD BODY CO.                      52
ICL CARGO TRAILER                       52
Name: MAKE, Length: 67, dtype: int64

In [185]:
df2['MAKE'] = df2['MAKE'].replace({'TOYOTA MOTOR COMPANY, LTD.' : 'TOYOTA' , 
                 'KIA MOTORS CORP' : 'KIA',
                'GENERAL MOTORS CORPORATION (GMC)': "GENERAL MOTORS CORP.",
                'TESLA MOTORS': 'TESLA',
                'LINCOLN-CONTINENTAL':'LINCOLN',
                'INTERNATIONAL TRAILER CORP.': 'INTERNATIONAL TRAILER CORPORATION',
                'PETERBILT MOTORS CO., DIVISION PACCAR, INC.' : 'PETERBILT MOTORS COMPANY (DIVISION OF PACCAR, INC.)',
                'FREIGHTLINER CORP.' : 'FREIGHTLINER CORPORATION',
                'ACURA (DIV. OF AMERICAN HONDA MOTOR CO.)' : 'ACURA',
                'KENWORTH MOTOR TRUCK CO.' : 'KENWORTH MOTOR TRUCK COMPANY'
                
                                  })

In [186]:
df2['VEHICLE_TYPE'].value_counts()

PASSENGER                         145952
SPORT UTILITY VEHICLE (SUV)        30957
VAN/MINI-VAN                       11895
PICKUP                              7897
TRUCK - SINGLE UNIT                 5482
UNKNOWN/NA                          3214
TRACTOR W/ SEMI-TRAILER             2818
OTHER                               2092
BUS OVER 15 PASS.                   1756
BUS UP TO 15 PASS.                   514
MOTORCYCLE (OVER 150CC)              447
TRACTOR W/O SEMI-TRAILER             325
SINGLE UNIT TRUCK WITH TRAILER       309
OTHER VEHICLE WITH TRAILER           277
MOTOR DRIVEN CYCLE                    27
MOPED OR MOTORIZED BICYCLE            26
Name: VEHICLE_TYPE, dtype: int64

In [187]:
VT_counts = df2['VEHICLE_TYPE'].value_counts()
over_20 = VT_counts[VT_counts > 20].index
df2 = df2[df2['VEHICLE_TYPE'].isin(over_20)]

In [188]:
df2['VEHICLE_TYPE'].value_counts()

PASSENGER                         145952
SPORT UTILITY VEHICLE (SUV)        30957
VAN/MINI-VAN                       11895
PICKUP                              7897
TRUCK - SINGLE UNIT                 5482
UNKNOWN/NA                          3214
TRACTOR W/ SEMI-TRAILER             2818
OTHER                               2092
BUS OVER 15 PASS.                   1756
BUS UP TO 15 PASS.                   514
MOTORCYCLE (OVER 150CC)              447
TRACTOR W/O SEMI-TRAILER             325
SINGLE UNIT TRUCK WITH TRAILER       309
OTHER VEHICLE WITH TRAILER           277
MOTOR DRIVEN CYCLE                    27
MOPED OR MOTORIZED BICYCLE            26
Name: VEHICLE_TYPE, dtype: int64

## Now to baseline modeling

In [189]:
#numeric_pipeline = Pipeline([('numnorm', StandardScaler())])

#ordinal_pipeline = Pipeline([
  #  ('ordenc', OrdinalEncoder(categories = [columns])),
  #  ('ordnorm', StandardScaler())])

nominal_pipeline = Pipeline([
    ('onehotenc', OneHotEncoder(sparse = False, handle_unknown = 'ignore')), 
    ('onehotnorm', MaxAbsScaler())])


In [190]:
cat_cols = ['MAKE', 'MOST_SEVERE_INJURY', 'DAMAGE', 'VEHICLE_TYPE']

In [191]:
ct = ColumnTransformer(
    [ ("nominalpipe", nominal_pipeline, cat_cols)])

In [192]:
pipe = Pipeline([('preprocess', ct),
                      ('model',
                       LogisticRegression(max_iter = 10000))])
pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE',
                                                   'VEHICLE_TYPE'])])),
                ('model', LogisticRegression(max_iter=10000))])

In [193]:
df2['target_mapped'].value_counts()

reckless driving                         87610
missed or misunderstood traffic signs    57137
nondefensive driving                     30800
issue with the driver or vehicle         21942
external variable                        16499
Name: target_mapped, dtype: int64

In [194]:
df3 = df2[['MAKE', 'MOST_SEVERE_INJURY', 'DAMAGE', 'VEHICLE_TYPE', 'target_mapped']]

In [195]:
df3['target_mapped'].value_counts()

reckless driving                         87610
missed or misunderstood traffic signs    57137
nondefensive driving                     30800
issue with the driver or vehicle         21942
external variable                        16499
Name: target_mapped, dtype: int64

In [196]:
X = df3.drop('target_mapped', axis=1)
y = df3['target_mapped']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE',
                                                   'VEHICLE_TYPE'])])),
                ('model', LogisticRegression(max_iter=10000))])

In [197]:
print(pipe.score(X_train,y_train))
pipe.score(X_test,y_test)

0.427431508849816


0.4315154913781018

In [198]:
preds = pipe.predict(X_test)

confusion_matrix(y_test, preds)

array([[   12,     6,   386,     0,  2958],
       [    9,    17,   681,     0,  3768],
       [    6,    20,  2382,     0,  8869],
       [   11,    11,   755,     1,  5219],
       [   13,    13,  1605,     0, 16056]])

# More cleaning and feature engineering

In [199]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213988 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    213988 non-null  object 
 1   SEX                            213988 non-null  object 
 2   EJECTION                       213988 non-null  object 
 3   INJURY_CLASSIFICATION          213988 non-null  object 
 4   DRIVER_VISION                  213988 non-null  object 
 5   PHYSICAL_CONDITION             213988 non-null  object 
 6   POSTED_SPEED_LIMIT             213988 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         213988 non-null  object 
 8   DEVICE_CONDITION               213988 non-null  object 
 9   WEATHER_CONDITION              213988 non-null  object 
 10  LIGHTING_CONDITION             213988 non-null  object 
 11  FIRST_CRASH_TYPE               213988 non-null  object 
 12  TRAFFICWAY_TYPE              

In [200]:
df2.columns

Index(['PERSON_TYPE', 'SEX', 'EJECTION', 'INJURY_CLASSIFICATION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION', 'POSTED_SPEED_LIMIT',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE',
       'CRASH_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION',
       'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'MOST_SEVERE_INJURY',
       'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR',
       'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'CRASH_DATE', 'UNIT_TYPE', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
       'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRE

In [201]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213988 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    213988 non-null  object 
 1   SEX                            213988 non-null  object 
 2   EJECTION                       213988 non-null  object 
 3   INJURY_CLASSIFICATION          213988 non-null  object 
 4   DRIVER_VISION                  213988 non-null  object 
 5   PHYSICAL_CONDITION             213988 non-null  object 
 6   POSTED_SPEED_LIMIT             213988 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         213988 non-null  object 
 8   DEVICE_CONDITION               213988 non-null  object 
 9   WEATHER_CONDITION              213988 non-null  object 
 10  LIGHTING_CONDITION             213988 non-null  object 
 11  FIRST_CRASH_TYPE               213988 non-null  object 
 12  TRAFFICWAY_TYPE              

In [202]:
df2['UNIT_TYPE'].value_counts()

DRIVER                 212017
PARKED                   1782
DRIVERLESS                168
NON-CONTACT VEHICLE        12
DISABLED VEHICLE            9
Name: UNIT_TYPE, dtype: int64

In [203]:
for column in df2[cat_cols].columns:
    print(f"Column: {column}")
    print(df2[column].value_counts())
    print("\n")

Column: MAKE
CHEVROLET                            27358
TOYOTA                               26193
FORD                                 25301
NISSAN                               18296
HONDA                                17191
                                     ...  
INTERNATIONAL COACH MANUFACTURING       56
MOTOR COACH INDUSTRIES, INC.            55
MASERATI                                55
BLUE BIRD BODY CO.                      52
ICL CARGO TRAILER                       52
Name: MAKE, Length: 67, dtype: int64


Column: MOST_SEVERE_INJURY
NO INDICATION OF INJURY     179187
NONINCAPACITATING INJURY     19679
REPORTED, NOT EVIDENT        10800
INCAPACITATING INJURY         4133
FATAL                          189
Name: MOST_SEVERE_INJURY, dtype: int64


Column: DAMAGE
OVER $1,500      142039
$501 - $1,500     51107
$500 OR LESS      20842
Name: DAMAGE, dtype: int64


Column: VEHICLE_TYPE
PASSENGER                         145952
SPORT UTILITY VEHICLE (SUV)        30957
VAN/MINI-VAN 

In [204]:
for column in df2[num_cols].columns:
    print(f"Column: {column}")
    print(df2[column].value_counts())
    print("\n")

Column: AGE
25.0     7017
27.0     6741
24.0     6711
26.0     6689
23.0     6636
         ... 
100.0       3
101.0       2
98.0        1
108.0       1
104.0       1
Name: AGE, Length: 105, dtype: int64


Column: STREET_NO
1600     1605
100      1509
800      1364
200      1305
300      1203
         ... 
5587        1
7571        1
9952        1
12407       1
8727        1
Name: STREET_NO, Length: 9788, dtype: int64


Column: VEHICLE_YEAR
2015.0    14801
2016.0    14124
2014.0    13706
2017.0    13389
2013.0    12665
          ...  
1965.0        1
1964.0        1
1960.0        1
1957.0        1
1952.0        1
Name: VEHICLE_YEAR, Length: 65, dtype: int64


Column: CRASH_HOUR
16    16593
15    16349
17    16090
14    14245
13    13000
18    12966
12    12929
8     12587
11    11893
9     10791
10    10585
7      9927
19     9237
20     7389
21     6576
22     5863
23     5166
6      4852
0      4073
1      3207
2      2792
5      2624
3      2234
4      2020
Name: CRASH_HOUR, dtype: i

In [205]:
target_dict

{'reckless driving': ['FOLLOWING TOO CLOSELY',
  'IMPROPER OVERTAKING/PASSING',
  'EXCEEDING SAFE SPEED FOR CONDITIONS',
  'EXCEEDING AUTHORIZED SPEED LIMIT',
  'IMPROPER TURNING/NO SIGNAL',
  'IMPROPER LANE USAGE',
  'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
  'DRIVING ON WRONG SIDE/WRONG WAYCELL PHONE USE OTHER THAN TEXTING',
  'TEXTING',
  'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',
  'DISTRACTION - FROM INSIDE VEHICLE'],
 'nondefensive driving': ['IMPROPER BACKING',
  'FAILING TO REDUCE SPEED TO AVOID CRASH',
  'TURNING RIGHT ON RED'],
 'missed or misunderstood traffic signs': ['DISREGARDING YIELD SIGN',
  'FAILING TO YIELD RIGHT-OF-WAY',
  'DISREGARDING TRAFFIC SIGNALS',
  'DISREGARDING OTHER TRAFFIC SIGNS',
  'DISREGARDING ROAD MARKINGS',
  'DISREGARDING STOP SIGN'],
 'issue with the driver or vehicle': ['DRIVING SKILLS/KNOWLEDGE/EXPERIENCE',
  'PHYSICAL CONDITION OF DRIVER',
  'EQUIPMENT - VEHICLE CONDI

In [206]:
df2['target_mapped'].value_counts()

reckless driving                         87610
missed or misunderstood traffic signs    57137
nondefensive driving                     30800
issue with the driver or vehicle         21942
external variable                        16499
Name: target_mapped, dtype: int64

# Now lets go back to modeling

In [207]:
numeric_pipeline = Pipeline([('numnorm', StandardScaler())])

#ordinal_pipeline = Pipeline([
  #  ('ordenc', OrdinalEncoder(categories = [columns])),
  #  ('ordnorm', StandardScaler())])

nominal_pipeline = Pipeline([
    ('onehotenc', OneHotEncoder(sparse = False, handle_unknown = 'ignore')), 
    ('onehotnorm', MaxAbsScaler())])

In [208]:
cat_cols = ['PERSON_TYPE', 'SEX', 'EJECTION', 'INJURY_CLASSIFICATION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 
       'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'REPORT_TYPE', 'CRASH_TYPE', 'DAMAGE', 
        'STREET_DIRECTION', 
       'STREET_NAME', 'MOST_SEVERE_INJURY',
        'UNIT_TYPE', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
        'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRECTION', 'MANEUVER', 'FIRST_CONTACT_POINT',
       'BAC_RESULT', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED']
num_cols = ['AGE', 'STREET_NO', 'VEHICLE_YEAR', 'CRASH_HOUR',
            'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'BEAT_OF_OCCURRENCE', 'POSTED_SPEED_LIMIT',
            'LATITUDE', 'LONGITUDE', 'OCCUPANT_CNT',
            'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
            'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
            'INJURIES_NO_INDICATION', 'NUM_UNITS', 'INJURIES_UNKNOWN']

In [209]:
ct = ColumnTransformer(
    [ ("nominalpipe", nominal_pipeline, cat_cols),
      ("numericpipe", numeric_pipeline, num_cols)
    ])

In [210]:
X.shape

(213988, 4)

In [211]:
X = df2.drop('target_mapped', axis=1)
y = df2['target_mapped']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [212]:
pipe = Pipeline([('preprocess', ct), 
                     ('model',
                        LogisticRegression(solver = 'liblinear'))])
pipe.fit(X_train, y_train)
pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['PERSON_TYPE', 'SEX',
                                                   'EJECTION',
                                                   'INJURY_CLASSIFICATION',
                                                   'DRIVER_VISION',
                                                   'PHYSICAL_CONDITION',
                                                   'TRAFFIC_CONTROL_DEVICE',
                        

In [213]:
print(pipe.score(X_train,y_train))
pipe.score(X_test,y_test)

0.6262398504585548


0.6166876956867143

In [214]:
pipe2 = Pipeline([
     ('preprocess', ct), 
     ('model', RandomForestClassifier(max_depth=20))
 ])

pipe2.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['PERSON_TYPE', 'SEX',
                                                   'EJECTION',
                                                   'INJURY_CLASSIFICATION',
                                                   'DRIVER_VISION',
                                                   'PHYSICAL_CONDITION',
                                                   'TRAFFIC_CONTROL_DEVICE',
                        

In [215]:
print(pipe2.score(X_train,y_train))
pipe2.score(X_test,y_test)

0.6362813248437409


0.5907519042945932

In [217]:
pipe1 = Pipeline([('preprocess', ct), 
                 ('model', XGBClassifier(eval_metric='logloss'))])

pipe1.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
print(pipe1.score(X_train,y_train))
pipe1.score(X_test,y_test)

In [ ]:

# Get the OneHotEncoder from the pipeline
one_hot_encoder = pipe.named_steps['preprocess'].named_transformers_['nominalpipe'].named_steps['onehotenc']

# Get the categorical feature names
categorical_feature_names = one_hot_encoder.get_feature_names(input_features=cat_cols)

# Combine the categorical and numerical feature names
all_feature_names = np.concatenate([categorical_feature_names, num_cols])

# Now compute the feature importances
feature_importance = pipe.named_steps['model'].coef_[0]
importances = pd.Series(feature_importance, index=all_feature_names)
importances = importances.abs().sort_values(ascending=False)
importances.head(30)

In [ ]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(num_cores)
multiprocessing.cpu_count()

In [ ]:
df2['PRIM_CONTRIBUTORY_CAUSE'].value_counts()